In [225]:
import pandas as pd
data = pd.read_csv('./ner_tagged.tsv', sep='\t')

In [226]:
data.rename(columns = {'Unnamed: 3':'Tags','-DOCSTART-':'Tokens','-X-':'X'}, inplace = True)

In [227]:
for i, row in data.reset_index().iterrows():
    if type(row.Tokens) == str:
        if row.Tokens.startswith("un") and "-N-" in row.Tags and row.Tokens != "undermounted":
            data.at[i,'Tags'] = row.Tags.replace("N-", "")
    if type(row.Tokens) == str:
        if row.Tokens.endswith("less") or row.Tokens.endswith("less.") and row.Tokens not in ["screw", "less"] and "-N-" in row.Tags:
            data.at[i,'Tags'] = row.Tags.replace("N-", "")

In [228]:
import numpy as np
is_negative = []
for i, row in data.reset_index().iterrows():
    if type(row.Tokens) == str:
        if "-N-" in row.Tags:
            is_negative.append(True)
            data.at[i, "Tags"] = row.Tags.replace("N-", "")
        else:
            is_negative.append(False)
    else:
        is_negative.append(np.NaN)

In [229]:
data["is_negative"] = is_negative

In [201]:
!pip install pandas==1.3.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.2
    Uninstalling pandas-1.4.2:
      Successfully uninstalled pandas-1.4.2



morphemes 1.0.9 requires pandas>=1.4.1, but you have pandas 1.3.0 which is incompatible.


In [230]:
import pandas
data.to_csv("C:\\Users\\andre\\Downloads\\MDS-CL\\Capstone\\NER\\NegativeAwareNER\\reannotated_val.csv")

In [195]:
neg_sents = []
pos_sents = []
current_sent = []
current_tags = []
contains_neg = False
non_neg = 0

for i, row in data.reset_index().iterrows():
    if type(row.Tags) == str:
        current_sent.append((row.Tokens, row.Tags, row.is_negative))
        if row.is_negative:
            contains_neg = True
    else:
        if contains_neg:
            neg_sents.append(current_sent)
        else:
            pos_sents.append(current_sent)
        current_sent = []
        contains_neg = False

In [196]:
for entry in neg_sents:
    print(entry)

[('Looking', 'O', False), ('for', 'O', False), ('pre', 'B-PRODUCT', False), ('workout', 'I-PRODUCT', False), ('Pump', 'I-PRODUCT', False), ('addict', 'I-PRODUCT', False), ('instead', 'O', False), ('of', 'O', False), ('Karbolyn', 'B-PRODUCT', True), ('Hydrate', 'I-PRODUCT', True), ('Wich', 'O', False), ('one', 'O', False), ('is', 'O', False), ('better?', 'O', False)]
[('Hello,', 'O', False), ('do', 'O', False), ('any', 'O', False), ('of', 'O', False), ('your', 'O', False), ('free', 'B-ATTRIBUTE', False), ('standing', 'I-ATTRIBUTE', False), ('tubs', 'B-PRODUCT', False), ('have', 'O', False), ('a', 'O', False), ('matte', 'B-TEXTURE', False), ('white', 'B-COLOUR', False), ('finish', 'O', False), ('instead', 'O', False), ('of', 'O', False), ('a', 'O', False), ('glossy', 'B-TEXTURE', True), ('finish?', 'O', False)]
[('Hi', 'O', False), ('there.', 'O', False), ("I'm", 'O', False), ('looking', 'O', False), ('to', 'O', False), ('buy', 'O', False), ('some', 'O', False), ('throw', 'B-PRODUCT', Fa

In [184]:
for entry in pos_sents:
    for word in entry:
        if word[0].startswith("un") and "-N-" in word[1]:
            print(entry)
        if word[0].endswith("less"):
            print(entry)

[('i', 'O'), ('need', 'O'), ('a', 'O'), ('48', 'B-SIZE'), ('inch', 'I-SIZE'), ('glass', 'B-MATERIAL'), ('sliding', 'B-PRODUCT'), ('goof', 'I-PRODUCT'), ('and', 'O'), ('a', 'O'), ('shower', 'B-PRODUCT'), ('pan', 'I-PRODUCT'), ('system', 'I-PRODUCT'), ('for', 'O'), ('500.99', 'B-PRICE'), ('or', 'I-PRICE'), ('less', 'I-PRICE')]
[('curbless', 'B-ATTRIBUTE'), ('shower', 'B-PRODUCT')]
[('twik', 'O'), ('white', 'B-COLOUR'), ('sleeveless', 'B-ATTRIBUTE'), ('button', 'B-ATTRIBUTE'), ('closure', 'I-ATTRIBUTE'), ('top', 'B-PRODUCT')]
[('Sleeveless', 'B-ATTRIBUTE'), ('sweater', 'B-PRODUCT')]
[('do', 'O'), ('you', 'O'), ('have', 'O'), ('the', 'O'), ('stainless', 'B-MATERIAL'), ('steel', 'I-MATERIAL'), ('shaker', 'B-PRODUCT'), ('cups', 'I-PRODUCT')]
[('Hi,', 'O'), ('do', 'O'), ('you', 'O'), ('sell', 'O'), ('fingerless', 'B-ATTRIBUTE'), ('gloves', 'B-PRODUCT'), ('please', 'O')]
[('hi,', 'O'), ('can', 'O'), ('I', 'O'), ('put', 'O'), ('a', 'O'), ('tankless', 'B-ATTRIBUTE'), ('toilet', 'B-PRODUCT'), ('i

## Categories

- bathroom (sink, mirror, shower, bathtub, shower curtains, squeegee, brush, toilet)
- blankets
- mirror
- clothing and accessories
- vanity/shelving/wardrobes
- kitchen (sink, dishes, island)
- glass doors
- gazebo
- supplements
- luggage
- furniture
- pool cue case
- barbecue

## Observations

- Difficult to locate and distinguish products and attributes even prior to attempting negation. Would it make sense to figure out what attributes likely to belong to a particular class of products are?
- Negation tends not to persist across sentence boundaries
- Punctuation is not separately tokenized
- Annotation of negative size is not perfectly consistent: in the 17th negative sentence, the height is negated ('15" height'), whereas in the 14th sentence, the comparative is negated ("bigger").
- In words where the negation is marked by a prefix, e.g. "unflavored" in "I'm looking for some unflavored mass gainer protein", the entire word is given a negative tag ("unflavored-B-N-ATTRIBUTE"). This seems like it could cause confusion, since in this case it implies that "unflavored" is not what is desired.
- The "direction" of negation is not always clear from the annotation. E.g., in "I found a vanity top I love but it has only one hole for taps and that doesn't suit our needs", "one hole" is negated, but it is not clear from that that more rather than fewer holes are desired.
- Lots of grammatical errors, e.g. "with not soy in it". Using word and character embeddings would help with this
- "doesn't need to be X-attribute" is annotated with a negation
- 

## Ways of indicating negation (incomplete):

- "don't want (any)"
- "no"
- "no more than"
- "less than"
- "un-"
- "without"
- "only"
- "-less"
- "instead of"
- "too" (what comes before is negated, e.g. in "pink is too light", "pink" is negated)
- "not too" (what comes after is negated, e.g. in "not too light", "light" is negated)
- "the website only gives the option of"
- "without X or Y" (two separately annotated entities separated by conjunction)